In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import scipy
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
import random
from deap import tools
from deap import base, creator
creator.create("FitnessMin", base.Fitness, weights = (-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

In [3]:
data = np.genfromtxt('data/cs268_copy.csv', delimiter=',')
# fiter the head, and filter deviceId
X = data[1:, 2:]
Y = data[1:, 1]
X = preprocessing.scale(X)
def err(Ytr, Ypr):
    y_t = np.array(Ytr)
    y_p = np.array(Ypr)
    return np.mean(Ypr.reshape(Ytr.shape) != Ytr)

In [16]:
POP_SIZE = 50  
CRS_RATE = 0.7
MUT_RATE = 0.02
NC = 15
NR = 15
FEATURENUM = len(X[0])
WA = 0.8
WF = 0.2
NGEN = 50
NSAMPLE = 5000

In [6]:
clf = SVC(C= 100, gamma = 4)
clf.fit(X[0:8000], Y[0:8000])

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=4, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [7]:
Ytep = clf.predict(X[10000:11000])
print "Accuracy for test data:{}".format(1 - err(Ytep, Y[10000:11000]))

Accuracy for test data:0.146


In [ ]:
dec = clf.decision_function([[1]])

In [36]:
import struct

def binary(val):
    return bin(val)

def evaluate(individual):
    #print(individual)
    clf = SVC()
    tmp = ''
    for i in range(NC):
        tmp = tmp + str(individual[i])
    clf.C = int(tmp, 2)*0.1**2
#     print "Cost = {}".format(clf.C)
    if (clf.C <= 0):
        return (1,)
    tmp = ''
    for i in range(NC, NC+NR):
        tmp = tmp + str(individual[i])
    clf.gamma = int(tmp, 2)*0.1**2
    #print clf.gamma
    feature = individual[NC+NR:]
    mask = []
    for i in range(len(feature)):
        if feature[i] is 0:
            mask.append(i)
    Xtrain2 = np.delete(X[0:NSAMPLE], mask, 1) 
    
    #for i in range(NSAMPLE):
    #    Xtrain2.append([a*int(b) for a, b in zip(Xtrain[i], feature)]) 
    clf.fit(Xtrain2, Y[0:NSAMPLE])
    Xtrain3 = np.delete(X[NSAMPLE + 1:NSAMPLE + 1000], mask, 1)
    res = (mean_squared_error(clf.predict(Xtrain3), Y[NSAMPLE + 1:NSAMPLE + 1000]),)
#     print "mse = {}, with C={} and gamma = {}".format(res, clf.C, clf.gamma)
    return res

#def val2pheno(val: float, minVal: float, maxVal: float, length: int) -> str:
#    return "{0:b}".format(minVal + (maxVal - minVal) * val / (2**length - 1))

In [37]:
IND_SIZE = NC + NR + FEATURENUM
toolbox = base.Toolbox()
toolbox.register("attribute", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attribute, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb = 0.1)
toolbox.register("select", tools.selTournament, tournsize = 3)
toolbox.register("evaluate", evaluate)

In [38]:
mse = []
# create an initial population of 300 individuals (where
# each individual is a list of integers)
pop = toolbox.population(n = POP_SIZE)

# Evaluate the entire population
fitnesses = list(map(toolbox.evaluate, pop))
for ind, fit in zip(pop, fitnesses):
   ind.fitness.values = fit

print("  Evaluated %i individuals" % len(pop))


for g in range(NGEN):
    print("-- Generation %i --" % g)
    
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    # Clone the selected individuals
    offspring = map(toolbox.clone, offspring)
    
    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        # cross two individuals with probability CXPB
        if random.random() < CRS_RATE:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values
            
    for mutant in offspring:
        # mutate an individual with probability MUTPB
        if random.random() < MUT_RATE:
            toolbox.mutate(mutant)
            del mutant.fitness.values
    
    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    print("  Evaluated %i individuals" % len(invalid_ind))
    fitnesses = map(toolbox.evaluate, invalid_ind)
    minMSE = 10000
    minGene = []
    # Evaluate the individuals with an invalid fitness
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
        #print ind
        if fit[0] < minMSE:
            minMSE = fit[0]
            minGene = ind
    # The population is entirely replaced by the offspring 
    pop[:] = offspring
    mse.append(min(fitnesses))

    # Gather all the fitnesses in one list and print the stats
    fits = [ind.fitness.values[0] for ind in pop]

    length = len(pop)
    mean = sum(fits) / length
    sum2 = sum(x*x for x in fits)
    std = abs(sum2 / length - mean**2)**0.5
    print "Min MSE for our dataset = {}".format(minMSE)
    print "Min genefor our dataset = {}".format(minGene)
    print("  Min %s" % min(fits))
    print("  Max %s" % max(fits))
    print("  Avg %s" % mean)
    print("  Std %s" % std)



  Evaluated 50 individuals
-- Generation 0 --
  Evaluated 36 individuals
Min MSE for our dataset = 20.3993993994
Min genefor our dataset = [0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0]
  Min 20.3993993994
  Max 25.1401401401
  Avg 21.3034234234
  Std 1.13645487082
-- Generation 1 --
  Evaluated 30 individuals
Min MSE for our dataset = 20.2252252252
Min genefor our dataset = [0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1]
  Min 20.2252252252
  Max 23.5135135135
  Avg 20.7472072072
  Std 0.492430036242
-- Generation 2 --
  Evaluated 38 individuals
Min MSE for our dataset = 20.2882882883
Min genefor our dataset = [0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1,

In [40]:

print int("011111100101010",2)
print int("101011111100101",2)

16170
22501


In [43]:
clf = SVC()
clf.C = 16170
clf.gamma = 22501
feature = [1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1]
#Xtrain = np.array(Xtrain)
mask = []
#print Xtrain[1:10]
for i in range(len(feature)):
    if feature[i] is 0:
        mask.append(i)
Xtr = np.delete(X, mask, 1) 
#Xtrain = np.delete(Xtrain, (1), axis=1)
#for i in range(5000):
#    Xtrain2.append([a*b for a, b in zip(Xtrain[i], feature)]) 
#print Xtrain[1:10]

clf.fit(Xtr[0:10000], Y[0:10000])
#print mean_squared_error(clf.predict(Xtrain2[5000:6000]), Ytrain[5000:6000])

#clf.get_params()
Ye = clf.predict(Xtr[10000:20000])
print err(Ye, Y[10000:20000])

0.854
